In [1]:
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler
import pandas as pd

def clean_data(df):
    # Remove rows where label == 'idle'
    df = df[df['label'] != 'idle']
    # Remove columns 'sequence', 'battery', and 'flags'
    df = df.drop(columns=['sequence', 'battery', 'flags'])
    # Remove rows where label is 'rest' or 'start'
    df = df[~df['label'].isin(['rest', 'start'])]
    # Remove the column 'timestamp'
    df = df.drop(columns=['timestamp'])
    # Define bandpass Butterworth filter parameters
    lowcut = 3  # Low cutoff frequency in Hz
    highcut = 30  # High cutoff frequency in Hz
    fs = 256  # Sampling frequency in Hz (assumed)
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    # Create Butterworth bandpass filter
    b, a = butter(N=4, Wn=[low, high], btype='band')
    # Apply filter to all channel columns
    filtered_data = df.filter(like='channel_').apply(lambda x: filtfilt(b, a, x), axis=0)
    # Combine filtered data back into the DataFrame
    df = pd.concat([filtered_data, df[['label']]], axis=1)
    # Scale all channel columns using StandardScaler
    scaler = StandardScaler()
    channel_columns = [col for col in df.columns if col.startswith('channel_')]
    df[channel_columns] = scaler.fit_transform(df[channel_columns])
    return df

# Loaded variable 'df' from URI: /home/francesco/git/esn_rpy/data/signal_eeg.csv
df = pd.read_csv(r'/home/francesco/git/esn_rpy/data/signal_eeg.csv')

df_clean = clean_data(df.copy())
df_clean.head()

,channel_0,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9,channel_10,channel_11,label
512,0.014870,0.000285,-0.023720,-0.007395,-0.018315,-0.024062,-0.050710,-0.026562,-0.097150,-0.031036,-0.032860,-0.067534,baseline_eyes_open
513,-0.007401,-0.057403,-0.024773,-0.198378,-0.024713,-0.240682,-0.334520,-0.310200,-0.269250,-0.311222,-0.248585,0.164328,baseline_eyes_open
514,-0.032122,-0.117036,-0.057000,-0.395715,-0.021913,-0.420198,-0.597169,-0.599840,-0.434638,-0.573828,-0.506030,0.243815,baseline_eyes_open
515,-0.058298,-0.173802,-0.133487,-0.593322,-0.006486,-0.531968,-0.819402,-0.886484,-0.578302,-0.802533,-0.810534,0.094050,baseline_eyes_open
516,-0.080471,-0.215234,-0.241614,-0.769771,0.015051,-0.561884,-0.987646,-1.144260,-0.676316,-0.985315,-1.117755,-0.253562,baseline_eyes_open
